In [ ]:
import json
import time
import numpy as np
import paho.mqtt.client as mqtt
import stable_baselines3
from stable_baselines3 import A2C
import os

/opt/conda/envs/conda_environment/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [ ]:
class MySimpleController:
    def __init__(self, radTrig=80, TinTrig=22):
        self.radTrig = radTrig
        self.TinTrig = TinTrig

    def compute_action(self, measurements: dict) -> dict:
        """
        measurements expected to include:
        'DNI', 'Tair_z1', 'Tair_z2', 'Tair_z4'
        """

        Rad = float(measurements.get('DNI', 0.0))

        Tin_Zone1  = float(measurements.get('Tair_z1', 0.0))
        Tin_Zone2      = float(measurements.get('Tair_z2', 0.0))
        Tin_Zone4   = float(measurements.get('Tair_z4', 0.0))

        # 🔽 Actuator names
        payload = {
            'Zona1_wall2_shade_FMU': 0.0,
            'Zona1_wall8_shade_FMU': 0.0,
            'Zona1_wall9_shade_FMU': 0.0,
            'Zona2_wall2_shade_FMU': 0.0,
            'Zona2_wall3_shade_FMU': 0.0,
            'Zona4_wall2_shade_FMU': 0.0
        }

        # 🛠 Same rule everywhere: if Tin > TinTrig and Rad > radTrig → 7.0 else 0.0

        # Zone 1 → 3 shades
        if Tin_Zone1 > self.TinTrig and Rad > self.radTrig:
            payload['Zona1_wall2_shade_FMU'] = 7.0
            payload['Zona1_wall8_shade_FMU'] = 7.0
            payload['Zona1_wall9_shade_FMU'] = 7.0

        # Zone 2 → 2 shades
        if Tin_Zone2 > self.TinTrig and Rad > self.radTrig:
            payload['Zona2_wall2_shade_FMU'] = 7.0
            payload['Zona2_wall3_shade_FMU'] = 7.0

        # Zone 4
        if Tin_Zone4 > self.TinTrig and Rad > self.radTrig:
            payload['Zona4_wall2_shade_FMU'] = 7.0

        return payload

In [ ]:
class MyRLController:
    def __init__(self, model_path="trained_a2c_model"):
        # Load the trained model
        self.model = A2C.load(model_path)
        
        # Define the exact order of observations the model was trained on
        self.observation_names = [
            'Tair_z1', 'Tair_z2', 'Tair_z4', 'T_ext', 'DNI', 'DistrictHeating', 
            'DistrictCooling', 'ShadeStatus_Zone1_Wall2', 'ShadeStatus_Zone1_Wall8',
            'ShadeStatus_Zone1_Wall9', 'ShadeStatus_Zone2_Wall2',
            'ShadeStatus_Zone2_wall3', 'ShadeStatus_Zone4_Wall2'
        ]
        
        # Define the action names in the order they appear in the model output
        self.action_names = [
            'Zona4_wall2_shade_FMU', 'Zona2_wall2_shade_FMU', 'Zona2_wall3_shade_FMU',
            'Zona1_wall2_shade_FMU', 'Zona1_wall8_shade_FMU', 'Zona1_wall9_shade_FMU'
        ]

    def compute_action(self, measurements: dict) -> dict:
        # Prepare the observation vector in the correct order
        # Missing values are defaulted to 0.0 to prevent crashes
        obs_list = [float(measurements.get(name, 0.0)) for name in self.observation_names]
        obs = np.array(obs_list, dtype=np.float32)

        # Predict the action using the loaded model
        # action will be an array of 0s and 1s (MultiDiscrete)
        action, _ = self.model.predict(obs, deterministic=True)

        # Map discrete actions {0, 1} to physical values {0.0, 7.0}
        # The notebook uses: 0 -> 0.0 and 1 -> 7.0
        physical_actions = np.where(action == 0, 0.0, 7.0)

        # Construct the return payload
        payload = {
            name: float(val) 
            for name, val in zip(self.action_names, physical_actions)
        }

        return payload

In [ ]:
# 📦 Buffer for last observation
obs_buffer = {}


# 🧾 MQTT callbacks (solo qui, non nel controller)
def on_connect(client, userdata, flags, rc):
    print(f"✅ Connected to MQTT broker (rc={rc})")
    client.subscribe("building/observation", qos=1)
    print("✅ Subscribed to topic: building/observation")

def on_obs(client, userdata, msg):
    global obs_buffer
    try:
        data = json.loads(msg.payload)
        obs_buffer.update(data)
    except Exception as e:
        print(f"Error parsing observation: {e}")


# 🧩 Setup MQTT client (usando le env var come nel controller originale)
mqtt_broker = os.getenv('MQTT_BROKER_HOST', 'mosquitto')
mqtt_port = int(os.getenv('MQTT_BROKER_PORT', 1883))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_obs
client.connect(mqtt_broker, mqtt_port)
client.loop_start()

# 🔧 Istance of the controller
controller = MySimpleController()

/tmp/ipykernel_777/613007330.py:24: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


✅ Connected to MQTT broker (rc=0)
✅ Subscribed to topic: building/observation


In [ ]:
# 🔁 Main loop
try:
    while True:
        if obs_buffer:
            # We directly pass the entire observation dictionary to the controller.
            # The controller is expected to return an action dict.
            action_dict = controller.compute_action(obs_buffer)

            # Publish the action to the MQTT broker.
            # You may switch to topic "/actuator" if you want to preserve the old behavior.
            client.publish("building/action", json.dumps(action_dict), qos=1)
            #print(f"[DEBUG] Action sent: {action_dict}")

            # Clear the observation buffer so that we wait for a new observation
            # before computing the next action.
            obs_buffer.clear()

        # Small sleep to avoid busy-looping while waiting for new messages.
        time.sleep(0.001)

except KeyboardInterrupt:
    # Graceful shutdown on Ctrl+C
    print("Stopping...")
    client.loop_stop()
    client.disconnect()

[DEBUG] Action sent: {'Kitchen_shade_FMU': 0.0, 'Bath_shade_1_FMU': 0.0, 'Room1_shade_1_FMU': 0.0, 'Room1_shade_2_FMU': 0.0, 'Bath2_Shade_FMU': 0.0, 'Living_shade_1_FMU': 0.0, 'Living_shade_2_FMU': 0.0, 'Room2_shade1_FMU': 0.0, 'Room2_shade_2_FMU': 0.0}
[DEBUG] Action sent: {'Kitchen_shade_FMU': 0.0, 'Bath_shade_1_FMU': 0.0, 'Room1_shade_1_FMU': 0.0, 'Room1_shade_2_FMU': 0.0, 'Bath2_Shade_FMU': 0.0, 'Living_shade_1_FMU': 0.0, 'Living_shade_2_FMU': 0.0, 'Room2_shade1_FMU': 0.0, 'Room2_shade_2_FMU': 0.0}
[DEBUG] Action sent: {'Kitchen_shade_FMU': 0.0, 'Bath_shade_1_FMU': 0.0, 'Room1_shade_1_FMU': 0.0, 'Room1_shade_2_FMU': 0.0, 'Bath2_Shade_FMU': 0.0, 'Living_shade_1_FMU': 0.0, 'Living_shade_2_FMU': 0.0, 'Room2_shade1_FMU': 0.0, 'Room2_shade_2_FMU': 0.0}
[DEBUG] Action sent: {'Kitchen_shade_FMU': 0.0, 'Bath_shade_1_FMU': 0.0, 'Room1_shade_1_FMU': 0.0, 'Room1_shade_2_FMU': 0.0, 'Bath2_Shade_FMU': 0.0, 'Living_shade_1_FMU': 0.0, 'Living_shade_2_FMU': 0.0, 'Room2_shade1_FMU': 0.0, 'Room2_sha